In [ ]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

##### Workflow of the project

1. Concatenating multiple files into a single dataframe
2. Feature Engineering
3. Data Visualization using Plotly Library
4. Forecasting the Revenue for the next year using LSTM

##### Questions to deal with:

1. Most Quantity Ordered Products
2. Orders Per Month
3. Orders Per City
4. Top 10 High Revenue Generating Products
5. Least 5 Revenue Generating Products
6. Revenue Generated Monthly
7. Revenue generated City-wise
8. Expected Approximate Revenue for the Next Year, month-wise

In [ ]:
# Concatenating all files using glob library

joined_files = os.path.join("Sales*.csv")

joined_list = glob.glob(joined_files)

df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [ ]:
# Def function for dataframe details:

def check_df(dataframe, head=5):
    print(" SHAPE ".center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(" TYPES ".center(70,'-'))
    print(dataframe.dtypes)
    print(" HEAD ".center(70,'-'))
    print(dataframe.head(head))
    print(" TAIL ".center(70,'-'))
    print(dataframe.tail(head))
    print(" MISSING VALUES ".center(70,'-'))
    print(dataframe.isnull().sum())
    print(" DUPLICATED VALUES ".center(70,'-'))
    print(dataframe.duplicated().sum())
    print(" DESCRIBE ".center(70,'-'))
    print(dataframe.describe())

check_df(df)

In [ ]:
# Dropping Null Values

df = df.dropna(how='all')

In [ ]:
# Value Counts

df['Quantity Ordered'].value_counts()

In [ ]:
# Value Counts

df['Price Each'].value_counts()

In [ ]:
# Dropping rows with unnecessary values from the features

df.drop(df[df['Quantity Ordered'] == 'Quantity Ordered'].index, inplace = True)
df.drop(df[df['Price Each'] == 'Price Each'].index, inplace = True)

In [ ]:
# Converting the datatypes to int and float of respective features

df['Quantity Ordered'] = df['Quantity Ordered'].astype(int)
df['Price Each'] = df['Price Each'].astype(float)

In [ ]:
# Converting Feature into Datetime feature

df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:
# Extracting City from the Address for ease of Analysis

df['City'] = df['Purchase Address'].str.split(',', expand = True)[1]

In [ ]:
# Creating new feature 'Revenue', which is Selling Price * Quantity

df['Revenue'] = df['Quantity Ordered'] * df['Price Each']

In [ ]:
# Extracting date, month, quarter from datetime feature

df['Date'] = df['Order Date'].dt.date
df['Month'] = df['Order Date'].dt.month
df['Quarter'] = df['Order Date'].dt.quarter

In [ ]:
# Mapping Month names for the feature

df['Month'] = df['Month'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'June',
                              7:'July', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})

In [ ]:
df.head()

In [ ]:
df.dtypes

### Exploratory Data Analysis

### Most Quantity Ordered Products

In [ ]:
quantity_ordered = df.groupby('Product')['Quantity Ordered'].sum().sort_values(ascending=False).reset_index()

fig1 = px.bar(quantity_ordered, x='Product', y='Quantity Ordered', text = 'Quantity Ordered',
             hover_data=['Product', 'Quantity Ordered'], color='Quantity Ordered')
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig1.update_layout(title_text='Most Ordered Products')

fig1.show()

### Orders Per Month

In [ ]:
order_per_month = df.groupby('Month')['Order ID'].nunique().reset_index()

colors = ['lightslategray',] * 12
colors[2] = 'crimson'

fig2 = go.Figure(data=[go.Bar(
    x= order_per_month['Month'],
    y= order_per_month['Order ID'],
    text = order_per_month['Order ID'],
    marker_color=colors
)])
fig2.update_traces(texttemplate="%{text:.2s}", textposition='outside')
fig2.update_layout(uniformtext_minsize=2, uniformtext_mode='hide')
fig2.update_layout(title_text='Highest Orders in a Month')

### Orders Per City

In [ ]:
order_per_city = df.groupby('City')['Order ID'].nunique().reset_index()

colors = ['lightslategray',] * 12
colors[7] = 'crimson'

fig3 = go.Figure(data=[go.Bar(
    x= order_per_city['City'],
    y= order_per_city['Order ID'],
    text = order_per_city['Order ID'],
    marker_color=colors
)])
fig3.update_traces(texttemplate="%{text:.2s}", textposition='outside')
fig3.update_layout(uniformtext_minsize=2, uniformtext_mode='hide')
fig3.update_layout(title_text='Highest Orders Per City')

### Top 10 High Revenue Generating Products

In [ ]:
high_revenue_products = df.groupby('Product')['Revenue'].sum().sort_values(ascending=False).head(10).reset_index()

fig4 = px.line(df, x='Product', y='Revenue', color='Product', markers=True, hover_data='Revenue')
fig4.update_layout(title = "Top Revenue Generated Products")
fig4.show()

### Least 5 Revenue Generating Products

In [ ]:
least_revenue_products = df.groupby('Product')['Revenue'].sum().sort_values(ascending =True).tail(5).reset_index()

fig5 = px.bar(least_revenue_products, x = 'Product', y = 'Revenue')
fig5.update_layout(title="Least 5 Revenue Generated Products")
fig5.show()

### Revenue Generated Monthly

In [ ]:
monthly_revenue = df.groupby('Month')['Revenue'].sum().sort_values(ascending=False).reset_index()

labels = monthly_revenue['Month']
values = monthly_revenue['Revenue']

fig6 = go.Figure(data=[go.Pie(labels=labels, values=values, text = monthly_revenue['Revenue'], pull=[0.2, 0, 0, 0])])
fig6.update_traces(texttemplate="%{text:.2s}", textfont_size=15, textposition='outside',
                  marker=dict(line=dict(color='#000000', width=2)))
fig6.update_layout(title_text='Revenue Generated Monthly')

### Revenue generated City-wise

In [ ]:
revenue_city = df.groupby('City')['Revenue'].sum().sort_values(ascending=False).reset_index()

fig7 = px.scatter(revenue_city, x="Revenue", y="City",
                 size="Revenue", color="City",
                 hover_name="City", log_x=False, size_max=70)
fig7.update_layout(title = 'Scatter of Revenue by City')
fig7.show()

### Products based on its price

In [ ]:
product_price = df.groupby('Product')['Price Each'].max().sort_values(ascending=False).head().reset_index()
product_price

In [ ]:
fig8 = px.histogram(df, x = 'Revenue', y = 'Quantity Ordered', color = 'City', marginal="box",
                  hover_data = df.columns)
fig8.show()

### Important Takeaways from the Analysis

##### 1. AAA Batteries is the most ordered product(31,017), followed by AA Batteries(27,635) and USB-C Charging Cable(23,975)
##### 2. December has the Highest Orders (24,004), followed by October(19,436)
##### 3. December has Highest Revenue of 4.6 Million, followed by October (3.7 Million) and April (3.4 Million)
##### 4. San Francisco has Highest Orders(42, 898)
##### 5. San Francisco has generated the Most Revenue of 8.2 Million, followed by Los Angeles(5.4 Million)
##### 6. Highest Revenue Generated Product are electronics -> Macbook Pro Laptop, iPhone, ThinkPad Laptop, Google Phone
##### 7. Least Revenue Generated Product are Lighting Chargin Cable, USB-C Charging Cable, Wired Headphones, AA Batteries

### Revenue Forecasting

In [ ]:
# Converting feature into datetime format

df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Creating a dataframe with only Date and Revenue features
# Setting the new dataframe with date index

new_df = df.groupby('Date')['Revenue'].sum().reset_index()
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df.set_index('Date', inplace =True)

In [ ]:
new_df

In [ ]:
revenue_data = new_df['Revenue'].values

In [ ]:
# Data scaling
scaler = MinMaxScaler()
revenue_data_scaled = scaler.fit_transform(revenue_data.reshape(-1, 1))

In [ ]:
# Prepare sequences of past time steps and corresponding target values
def prepare_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

In [ ]:
n_steps = 30  # Number of past time steps to use for predictions
x_new, y_new = prepare_sequences(revenue_data_scaled, n_steps)

In [ ]:
# Splitting the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=30, shuffle=False)

# Reshape the data to fit the LSTM model (samples, timesteps, features)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=150, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

In [ ]:
# Forecast for the next 12 months
future_data_scaled = revenue_data_scaled[-n_steps:].reshape(1, n_steps, 1)  # Use the last n_steps as input for forecasting
future_data = []

In [ ]:
for _ in range(12):
    prediction = model.predict(future_data_scaled)
    future_data_scaled = np.append(future_data_scaled[:, 1:, :], prediction.reshape(1, 1, 1), axis=1)
    future_data.append(prediction[0, 0])

In [ ]:
# Inverse transform the scaled data to get the actual revenue values
forecasted_revenue = scaler.inverse_transform(np.array(future_data).reshape(-1, 1))

In [ ]:
# Create a DataFrame to store the forecasted values with the appropriate datetime index
forecast_dates = pd.date_range('2020-01-02', periods=12, freq='M')
forecast_df = pd.DataFrame(forecasted_revenue, index=forecast_dates, columns=['revenue_forecast'])

In [ ]:
# Print the forecasted DataFrame
print(forecast_df)